In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gmean

In [ ]:
def df_from_csv(file):
    df = pd.read_csv(file, header = 1)
    return df

In [ ]:
def remove_negatives(df):
    #df is a row from a multi-cuty timeseries dataframe
    for i, d in enumerate(df):
        if df[i]<=0:
            df[i]=0
        else:
            df[i]=df[i]
    return df

In [ ]:
def find_seq_maxs(df):
    #df is a row from a multi-cuty timeseries dataframe
    peaks = 0
    seq_max = 0
    maximums = []
    for i, d in enumerate(df):
        if i < len(df) - 1:
            if df[i] == 0:
                maximums.append(seq_max) #when a sequence of non-zeros ends capture recorded maximum of the sequence
                seq_max = 0
                if df[i+1] != 0:
                    peaks += 1
            elif df[i] > 0:
                if df[i+1] != 0:
                    maximum = max(df[i], df[i+1])
                    if maximum > seq_max:
                        seq_max = maximum #record new sequence maximum
                    else:
                        pass
                else:
                    if df[i-1] == 0:
                        seq_max = df[i]
            else:
                i += 1
        else:
            seq_max = max(seq_max,df[-1])
            maximums.append(seq_max)
    maximums = [i for i in maximums if i != 0]
    return maximums, peaks

In [ ]:
def transitions_count(df): #probability FF
    #df is a row from a multi-cuty timeseries dataframe
    FF = 0
    FAll = 0
    maximums = []
    for i, d in enumerate(df):
        if i < len(df) - 1:
            if df[i] == 0:
                pass
            elif df[i] > 0:
                FAll += 1
                if df[i+1] != 0:
                    FF += 1
            else:
                i += 1
        else:
            pass
    return FF, FAll

In [ ]:
#Open file and reorder
df=pd.read_csv('UrbanWloupe_storage.csv', header=None).drop([0]).T
df.columns=df.iloc[0]
df=df[1:].rename(columns={"area_id": "date"})
#df=df.rename(columns={"area_id": "date"})
#df=df[1:]
df['date'] =  pd.to_datetime(df['date'], dayfirst=True)
df.index=df['date']
df=df.drop(axis=1, labels='date').astype(float, errors = 'raise')
df


In [ ]:
#Define simulated period and calculate the relevant timeseries parameters
y_start=1981
y_end=2010
stor_df=df.loc[str(y_start):str(y_end)]
years=np.arange(y_start, y_end+1, 1, dtype=None)
average_min=[]
range=[]
for col in stor_df.columns:
    print(col)
    cit = stor_df[str(col)] 
    range.append(np.ptp(cit,axis=0)) 
    minimums=[]
    lowflow_month=[]
    for y in years:
        ins=cit.loc[str(y)]
        y_min=min(ins)
        y_max=max(ins)
        lowflow_month.append(np.isin(ins,y_min))
    lowestflows=np.where(sum(lowflow_month)==max(sum(lowflow_month)))
    if lowestflows[0][0]<=6:
        cit_wy_series = cit.index.year.where(cit.index.month-1 <= (lowestflows[0][0]+5), cit.index.year+1)
    else:
        cit_wy_series = cit.index.year.where(cit.index.month-1+12 > (lowestflows[0][0]+5), cit.index.year-1)
    frame = { 'WaterYear':cit_wy_series ,'Storage': cit}
    cit_wy = pd.DataFrame(frame)
    for y in years:
        select_wy=np.where(cit_wy.WaterYear==y)
        ins2=cit_wy.iloc[select_wy[0][0]:select_wy[0][-1]+1,1]
        y_min_wy=min(ins2)    
        minimums.append(y_min_wy)
    average_min.append(np.mean(minimums))
wg_raw=average_min-stor_df


   

In [ ]:
#Calculate water gap from interim water gap with negatives
wg=[]
for col in stor_df.columns:
    citwg = wg_raw[str(col)]
    wg_i=remove_negatives(citwg)
    wg.append(wg_i)
    
wg_df=pd.DataFrame(data=np.transpose(wg),index=stor_df.index, columns=stor_df.columns)

In [ ]:
#Generate a normalizer for the WG and normalize WG
normalizer=np.array(range)
wg_norm=np.divide(np.array(wg_df),np.array(normalizer))

In [ ]:
#Find the count of failed months and the number of transitions from failed month to failed month
wg_count=[]
wg_sev=[]
Suc=[]
Fail=[]
FailFail=[]
FailAll=[]
pers_pap=[]

for cit in wg: #cit is a timeseries of watergaps
    Fail.append(np.count_nonzero(cit))
    Suc.append(len(cit)-np.count_nonzero(cit))
    process=find_seq_maxs(cit)
    peaktotal=sum(process[0])
    wg_count.append(process[1])
    process2=transitions_count(cit)
    FailFail=process2[0]
    FailAll=process2[1]
    pers_pap.append(np.divide(np.array(FailFail),np.array(FailAll)))
    wg_sev=peaktotal/wg_count
total=len(wg[0])
wg_norm_sev=wg_sev/normalizer
wg_freq=1-(np.array(Suc))/total
wg_pers=pers_pap






In [ ]:
#Calculate the hazard score based on the geometric mean of the 3 parameters: Severity, Frequency, Persistence
hazard=gmean([wg_norm_sev, wg_freq, wg_pers],axis=0)

In [ ]:
hazard_df=pd.DataFrame(data=[wg_norm_sev, wg_freq, wg_pers, hazard], index=['Severity', 'Frequency', 'Persistence', 'Hazard'],columns=stor_df.columns[0:len(hazard)])
hazard_df


In [ ]:
print(hazard_df['8675'])
print(hazard_df['512'])
print(hazard_df['3268'])
print(hazard_df['1303'])

In [ ]:
wg_norm_df=wg_df/normalizer
wg_norm_df['Type']='Water Gap'
wg_norm_df.to_csv('wgn.csv')
wg_norm_df

In [ ]:
stor_df['Type']='Storage'
stor_df.to_csv('stor.csv')
wg_df['Type']='Water Gap'
wg_df.to_csv('wg.csv')
hazard_df.to_csv('hazard.csv')

In [ ]:
# Define Plot
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Water gap in m', dpi=100):
    plt.figure(figsize=(16,3), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

In [ ]:
#Draw inspect plot by year
city=8675

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
plotdf=pd.DataFrame(
    {'date' : df.index,
    'ws' : df[str(city)],
    'wg' : wg_df[str(city)]})

definesize=(15,3)
fig = plt.figure()
ax = fig.add_subplot(111)
dateticks = pd.date_range('2000', '2007', freq=pd.DateOffset(years=1))
plotdf.plot(ax=ax, layout=(1,2), color='#56B4E9', xlim=(plotdf.date[239], plotdf.date[-50]),x='date', y='wg', grid=0, figsize=definesize, ylabel='Water Gap [m]',rot=45)
ax.xaxis.set_ticks(dateticks);
ax.xaxis.set_ticklabels(dateticks.strftime('%Y'));
ax.get_legend().remove();
image_name = 'wg_zoom.png'
image_format= 'png'
#fig.savefig(image_name, format=image_format, dpi=300)



In [ ]:
city=8675
storsample=stor_df[str(city)]
wgsample=wg_df[str(city)]
plot_df(storsample, x=storsample.index, y=storsample, title='Storage', ylabel='Storage in m') 
plot_df(wgsample, x=wgsample.index, y=wgsample, title='Water Gap', ylabel='Water Gap in m') 

%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':300})
